<a href="https://colab.research.google.com/github/cristiandarioortegayubro/BDS/blob/main/modulo.01/bds_web_scraping_001_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<p align="center">
<img src="https://github.com/cristiandarioortegayubro/BDS/blob/main/images/Logo%20BDS%20Horizontal%208.png?raw=true">
</p>


<p align="center">
<img src="https://github.com/cristiandarioortegayubro/BDS/blob/main/images/Logo%20Pandas.png?raw=true">
</p>


 # **<font color="DeepPink">Web Scraping en Administración Tributaria Mendoza</font>**

<p align="center">
<img src="https://atm.mendoza.gov.ar/wp-content/uploads/2023/01/Frente-Atm.png">
</p>



https://atm.mendoza.gov.ar/portalatm/zoneBottom/datosInteres/recaudacion/recaudacion_impuesto_ingresos.jsp


<p align="justify">
👀 En la página web de la Administración Tributaria Mendoza, podemos ver la recaudación por ejercicios fiscales del Impuesto sobre los Ingresos Brutos. Lo que vamos a hacer, es tomar esos datos y generar nuestro <code>DataFrame</code> para gráficar.</p>

 # **<font color="DeepPink">Habilitando bibliotecas requeridas</font>**

 ## **<font color="DeepPink">Para análisis de datos</font>**

In [1]:
import numpy as np
import pandas as pd

 ## **<font color="DeepPink">Para web scraping</font>**

In [2]:
import lxml
import bs4
import html5lib

 ## **<font color="DeepPink">Configuraciones del DataFrame</font>**

In [3]:
pd.options.display.precision = 2
pd.options.display.float_format = "{:.2f}".format

 ## **<font color="DeepPink">Para graficos</font>**

In [4]:
import plotly.express as px
import plotly.graph_objects as go

 # **<font color="DeepPink">Obtención de datos y generación del DataFrame</font>**

In [5]:
data = "https://atm.mendoza.gov.ar/portalatm/zoneBottom/datosInteres/recaudacion/recaudacion_impuesto_ingresos.jsp"
lista = pd.read_html(data)
df = lista[0]
df.columns = df.loc[1]
df.columns.name = None
df = df.loc[2:13]
df = df.dropna(axis=1, how="all")
df.set_index("Meses", inplace=True)
df = df.astype("float")
Meses = {"Enero":"1", "Febrero":"2", "Marzo":"3", "Abril":"4", "Mayo":"5", "Junio":"6", "Julio":"7", "Agosto":"8", "Septiembre":"9", "Octubre":"10", "Noviembre":"11", "Diciembre":"12"}
df.index = df.index.map(Meses)
lista = []
for i, j in enumerate(df.columns):
  datos = pd.DataFrame(df[j])
  datos = datos.rename(columns={j:"iibb"})
  datos["Año"] = 2000+i
  datos["Año"] = datos.Año.astype("str")
  datos = datos[["Año","iibb"]]
  lista.append(datos)
iibb = pd.concat(lista)
iibb.index.name = "Mes"
iibb.reset_index(inplace=True)
iibb["Fecha"] = iibb.Año + iibb.Mes
iibb.Fecha = pd.to_datetime(iibb.Fecha, format="%Y%m")


# **<font color="DeepPink">Gráfico</font>**


<p align="justify">
👀 En este caso, el eje $X$ posee datos de tipo fecha. Por ese motivo, se genera una serie de tiempo, y el parametro <code>color</code> lo que hace es separar esa serie de tiempo por los distintos ejercicios fiscales, pero como tenemos ese eje con datos de tipo fecha, podemos agregar los botones con los selectores de meses que se quieren visualizar, desde el ultimo dato, hacia atrás.</p>

In [10]:
fig = px.line(iibb.query("Año > '2002' & Año < '2005'"),
              x="Fecha",
              y="iibb",
              color="Año",
              template="gridon",
              title="Ingresos Brutos",
              markers=True,
              labels={"iibb":"Recaudación"})

fig.update_xaxes(rangeselector=dict(buttons=list([
    dict(count=3, label="3m", step="month", stepmode="backward"),
    dict(count=6, label="6m", step="month", stepmode="backward"),
    dict(count=9, label="9m", step="month", stepmode="backward"),
    dict(count=12, label="12m", step="month", stepmode="backward"),
    dict(count=15, label="15m", step="month", stepmode="backward"),
    dict(count=18, label="18m", step="month", stepmode="backward"),
    dict(count=21, label="21m", step="month", stepmode="backward"),
    dict(label = "Máx.", step="all")])))

fig.show()